# Final Project: Data Cleaning

## Importing Libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import *
import calendar
from tqdm import tqdm_notebook as tqdm

In [32]:
from google.colab import drive 
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing Datasets

### Load Data

In [33]:
data_path = "/content/drive/My Drive/2021/6.859/FinalProject/obesity-cleaned.csv"
data = pd.read_csv(data_path)
data.head()

,Unnamed: 0,Country,Year,Obesity (%),Sex
0,0,Afghanistan,1975,0.5 [0.2-1.1],Both sexes
1,1,Afghanistan,1975,0.2 [0.0-0.6],Male
2,2,Afghanistan,1975,0.8 [0.2-2.0],Female
3,3,Afghanistan,1976,0.5 [0.2-1.1],Both sexes
4,4,Afghanistan,1976,0.2 [0.0-0.7],Male


In [34]:
data = data[['Country', 'Year','Obesity (%)','Sex']] 
data.head()

,Country,Year,Obesity (%),Sex
0,Afghanistan,1975,0.5 [0.2-1.1],Both sexes
1,Afghanistan,1975,0.2 [0.0-0.6],Male
2,Afghanistan,1975,0.8 [0.2-2.0],Female
3,Afghanistan,1976,0.5 [0.2-1.1],Both sexes
4,Afghanistan,1976,0.2 [0.0-0.7],Male


In [35]:
data.drop(data[data['Sex'] == 'Both sexes'].index, inplace = True)

In [36]:
data.rename(columns = {'Obesity (%)' : 'Obesity'}, inplace = True)

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16380 entries, 1 to 24569
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  16380 non-null  object
 1   Year     16380 non-null  int64 
 2   Obesity  16380 non-null  object
 3   Sex      16380 non-null  object
dtypes: int64(1), object(3)
memory usage: 639.8+ KB


In [38]:
count = 0
null_countries = set()
for i, row in data.iterrows():
  cur = row['Obesity']
  if cur == 'No data':
    null_countries.add(row['Country'])
    count += 1
print(null_countries)

{'Monaco', 'Sudan', 'San Marino', 'South Sudan'}


In [39]:
for nc in null_countries:
  data.drop(data[data['Country'] == nc].index, inplace = True)

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16044 entries, 1 to 24569
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Country  16044 non-null  object
 1   Year     16044 non-null  int64 
 2   Obesity  16044 non-null  object
 3   Sex      16044 non-null  object
dtypes: int64(1), object(3)
memory usage: 626.7+ KB


In [41]:
for i, row in data.iterrows():
  cur = row['Obesity']
  data.at[i,'Obesity'] = cur[:cur.index('[')-1]

In [42]:
data.head()

,Country,Year,Obesity,Sex
1,Afghanistan,1975,0.2,Male
2,Afghanistan,1975,0.8,Female
4,Afghanistan,1976,0.2,Male
5,Afghanistan,1976,0.8,Female
7,Afghanistan,1977,0.2,Male


### Add Country Info

In [43]:
!pip install countryinfo
from countryinfo import CountryInfo

In [44]:
data['Region'] = None
data['Population'] = 0

In [45]:
region_dict = dict()
pop_dict = dict()

In [46]:
country = CountryInfo('Democratic Republic of Timor-Leste')
print(country.region())
print(country.population())

Asia
1172390


In [47]:
data['Country'] = data['Country'].replace(['Gambia'], 'The Gambia')
data.drop(data[data['Country'] == 'Andorra'].index, inplace = True)
data['Country'] = data['Country'].replace(['Côte d\'Ivoire'], 'Ivory Coast')
data.drop(data[data['Country'] == 'Republic of North Macedonia'].index, inplace = True)
data['Country'] = data['Country'].replace(['Brunei Darussalam'], 'Brunei')
data['Country'] = data['Country'].replace(['Congo'], 'Democratic Republic of the Congo')
data['Country'] = data['Country'].replace(['Bolivia (Plurinational State of)'], 'Bolivia')
data.drop(data[data['Country'] == 'Montenegro'].index, inplace = True)
data['Country'] = data['Country'].replace(['Sudan (former)'], 'Sudan')
data['Country'] = data['Country'].replace(['Viet Nam'], 'Vietnam')
data['Country'] = data['Country'].replace(['Bahamas'], 'the Bahamas')
data['Country'] = data['Country'].replace(['Venezuela (Bolivarian Republic of)'], 'Venezuela')
data['Country'] = data['Country'].replace(['Cabo Verde'], 'Republic of Cabo Verde')
data.drop(data[data['Country'] == 'Sao Tome and Principe'].index, inplace = True)
data['Country'] = data['Country'].replace(['Iran (Islamic Republic of)'], 'Iran')
data['Country'] = data['Country'].replace(['Czechia'], 'Czech Republic')
data['Country'] = data['Country'].replace(['United Kingdom of Great Britain and Northern Ireland'], 'United Kingdom')
data.drop(data[data['Country'] == 'Myanmar'].index, inplace = True)
data['Country'] = data['Country'].replace(['Micronesia (Federated States of)'], 'Federated States of Micronesia')
data['Country'] = data['Country'].replace(['Eswatini'], 'Kingdom of Swaziland')
data['Country'] = data['Country'].replace(['Timor-Leste'], 'Democratic Republic of Timor-Leste')

In [48]:

null_country = set()
for i, row in data.iterrows():
  cur_country = row['Country']
  if cur_country in region_dict:
    region = region_dict[cur_country]
    pop = pop_dict[cur_country]
  else:
    country = CountryInfo(cur_country)
    try:
      region = country.region()
      pop = country.population()
    except:
      region = None
      pop = -1
      null_country.add(cur_country)
    
    region_dict[cur_country] = region
    pop_dict[cur_country] = pop
  
  data.at[i,'Region'] = region
  data.at[i,'Population'] = pop

print(null_country)

set()


In [49]:
data['Obesity'] = pd.to_numeric(data['Obesity'])

In [50]:
data.describe()

,Year,Obesity,Population
count,15624.000000,15624.000000,1.562400e+04
mean,1995.500000,12.457188,3.831938e+07
std,12.121306,10.737851,1.404538e+08
min,1975.000000,0.100000,1.613000e+03
25%,1985.000000,3.600000,2.024904e+06
50%,1995.500000,10.400000,8.397815e+06
75%,2006.000000,18.200000,2.602310e+07
max,2016.000000,63.300000,1.367110e+09


In [51]:
data.head()

,Country,Year,Obesity,Sex,Region,Population
1,Afghanistan,1975,0.2,Male,Asia,26023100
2,Afghanistan,1975,0.8,Female,Asia,26023100
4,Afghanistan,1976,0.2,Male,Asia,26023100
5,Afghanistan,1976,0.8,Female,Asia,26023100
7,Afghanistan,1977,0.2,Male,Asia,26023100


In [52]:
set(region_dict.values())

{'Africa', 'Americas', 'Asia', 'Europe', 'Oceania'}

## To Dict for JSON

In [53]:
dict_data = dict()

In [54]:
for i, row in data.iterrows():
  if row['Year'] not in dict_data:
    dict_data[row['Year']] = []
  
  cur_dict = dict()
  cur_dict['Country'] = row['Country']
  cur_dict['Sex'] = row['Sex']
  cur_dict['Obesity'] = row['Obesity']
  cur_dict['Region'] = row['Region']
  cur_dict['Population'] = row['Population']
  dict_data[row['Year']].append(cur_dict)


In [55]:
for year in dict_data:
  for i in range(len(dict_data[year])):
    if i % 2 == 0:
      assert dict_data[year][i]['Country'] == dict_data[year][i+1]['Country']
      average = (dict_data[year][i]['Obesity'] + dict_data[year][i+1]['Obesity']) / 2
      dict_data[year][i]['AverageObesity'] = average
      dict_data[year][i+1]['AverageObesity'] = average


In [56]:
import json

In [57]:
dict_to_json = json.dumps(dict_data)

In [58]:
dict_to_json

Output hidden; open in https://colab.research.google.com to view.